In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [4]:
def to_tensor(X_train, X_test, y_train, y_test):
    X_train = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
    X_test = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

    return X_train, X_test, y_train, y_test


In [5]:
X, y = df.copy().drop('Outcome', axis=1), df.Outcome
X_train, X_test, y_train, y_test = to_tensor(*train_test_split(X, y, test_size=0.3))

In [6]:
class NeuralNetwork(keras.Model):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = keras.Sequential([
            keras.layers.Dense(16, activation='relu', input_shape=(8,)),
            keras.layers.Dense(8, activation='relu'),
            keras.layers.Dense(4, activation='relu'),
            keras.layers.Dense(1, activation='sigmoid')
        ])

    def call(self, x):
        return self.linear_relu_stack(x)

In [7]:
model = NeuralNetwork()
print(model)

In [8]:
loss_fn = keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.RMSprop()

In [9]:
@tf.function
def train_step(X, y, model, loss_fn, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [10]:
@tf.function
def test_step(X, y, model, loss_fn):
    predictions = model(X)
    loss = loss_fn(y, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(predictions), y), tf.float32))
    return loss, accuracy

In [11]:
epochs = 1000
for t in range(epochs):
    train_loss = train_step(X_train, y_train, model, loss_fn, optimizer)
    test_loss, test_accuracy = test_step(X_test, y_test, model, loss_fn)

    if t % 100 == 0:
        print(f"Epoch {t+1}\n-------------------------------")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

print("Done!")

Epoch 1
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 101
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 201
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 301
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 401
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 501
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 601
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 701
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 801
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test Accuracy: 0.6580
Epoch 901
-------------------------------
Train Loss: 5.4289
Test Loss: 5.2752, Test